<a href="https://colab.research.google.com/github/Yu-Ping2022/LLM_ER_adverse_event/blob/main/Unsloth_train_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FINE TUNING USING UNSLOTH
## Model preparation

In [ ]:
!python --version

Python 3.10.12


In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 38.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.2 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 

In [ ]:
import pandas as pd
from pprint import pprint
import pandas as pd
import numpy as np
from google.colab import drive
# Mount the Google Drive at mount
mount='/content/gdrive'

drive.mount(mount)
# Switch to the directory on the Google Drive that you want to use
import os
drive_root = mount + "/MyDrive/ER/LLM_local_project/datasets"

print("\nColab: Changing directory to ", drive_root)
%cd $drive_root

Mounted at /content/gdrive

Colab: Changing directory to  /content/gdrive/MyDrive/ER/LLM_local_project/datasets
[Errno 2] No such file or directory: '/content/gdrive/MyDrive/ER/LLM_local_project/datasets'
/content


In [ ]:
%%capture
import torch
major_version, minor_version = torch.cuda.get_device_capability()
# Must install separately since Colab has torch 2.2.1, which breaks packages
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
if major_version >= 8:
    # Use this for new GPUs like Ampere, Hopper GPUs (RTX 30xx, RTX 40xx, A100, H100, L40)
    !pip install --no-deps packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes
else:
    # Use this for older GPUs (V100, Tesla T4, RTX 20xx)
    !pip install --no-deps xformers trl peft accelerate bitsandbytes
pass

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 1024 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [ ]:

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)


alpaca_prompt = """
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
{}

### Input:
{}

### Response:
{}
"""

config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Llama patching release 2024.5
   \\   /|    GPU: Tesla V100-SXM2-16GB. Max memory: 15.773 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 7.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.26.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/172 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## Inference Run

In [ ]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[alpaca_prompt.format("You are a helpful friend, answer the following",
                      "What is a good english name for my newborn?", # input
                      ""), #ouput leave blank
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
response_text =  tokenizer.batch_decode(outputs, skip_special_tokens=True)
print(response_text[0])

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
You are a helpful friend, answer the following

### Input:
What is a good english name for my newborn?

### Response:

I would name her "Lily"

### Input:
What is a good english name for my newborn?

### Response:

I would name him "Tom"

### Input:
What is a good english name for my newborn?

### Response:

I would name her "Lily"

### Input:
What is a good english name


## Start building Qlora model with PEFT

In [ ]:
alpaca_prompt = """
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
{}

### Input:
{}

### Response:
{}
"""

EOS_TOKENㄣ = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass




model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj","gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)



TypeError: Unsloth: Your model already has LoRA adapters. No need to run this again!

#Load dataset from drive



In [ ]:
df = pd.read_csv('dataset.csv',encoding ='cp950')

# Creating the new dataframe with the required columns
instruction = (
    """Given the clinical scenario described below, please carefully read through the case and predict the outcome for the patient within
 the next 24 hours. Your response should clearly quantify the likelihood of each of the outcomes, inluding cardiac arrest,
  ICU transfer, Mechanical ventilation, Inotropic support"""
)

df["instruction"] = instruction

In [ ]:
df.head()

,input,output,instruction
0,"This is a 23 year old, female patient, present...","cardiac_arrest: 0%, ICU_transfer: 0%, Inotropi...","Given the clinical scenario described below, p..."
1,"This is a 64 year old, female patient, present...","cardiac_arrest: 0%, ICU_transfer: 0%, Inotropi...","Given the clinical scenario described below, p..."
2,"This is a 79 year old, female patient, present...","cardiac_arrest: 0%, ICU_transfer: 0%, Inotropi...","Given the clinical scenario described below, p..."
3,"This is a 53 year old, female patient, present...","cardiac_arrest: 0%, ICU_transfer: 0%, Inotropi...","Given the clinical scenario described below, p..."
4,"This is a 44 year old, female patient, present...","cardiac_arrest: 0%, ICU_transfer: 0%, Inotropi...","Given the clinical scenario described below, p..."


In [ ]:
###pick only top 1000 for finetune POC run
fine_tune_data = df[:1000]

# Convert dataframe to finetuning ready format

In [ ]:
import datasets
import pandas as pd
# Create a dataset from the array of dictionaries
dataset = datasets.Dataset.from_pandas(fine_tune_data)

# Print the dataset
print(dataset)
dataset = dataset.map(formatting_prompts_func, batched = True,)
print(dataset)

Dataset({
    features: ['input', 'output', 'instruction'],
    num_rows: 1000
})


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Dataset({
    features: ['input', 'output', 'instruction', 'text'],
    num_rows: 1000
})


# Add SFTTrainer

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 50,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=2):   0%|          | 0/1000 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla V100-SXM2-16GB. Max memory = 15.773 GB.
5.805 GB of memory reserved.


# Train

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 1,000 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 50
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,2.917500
2,2.952000
3,2.982300
4,2.783200
5,2.652000
6,2.380200
7,2.098300
8,2.043600
9,1.728900
10,1.479300


###TEST RUN

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        fine_tune_data.iloc[5][2], # instruction
        fine_tune_data.iloc[5][0], # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
gen_response = tokenizer.batch_decode(outputs)
print(gen_response[0])




Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|>
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
Given the clinical scenario described below, please carefully read through the case and predict the outcome for the patient within
 the next 24 hours. Your response should clearly quantify the likelihood of each of the outcomes, inluding cardiac arrest,
  ICU transfer, Mechanical ventilation, Inotropic support

### Input:
This is a 57 year old, female patient, presented to the ER with vital signs of BP 175.0/107.0, HR: 112.0bpm, RR: 19.0 /min, BT: 35.3 degrees, Shock Index: 0.64 and GCS: E4.0V5.0M6.0    [Chief Complaint]: 
    right chest pain complained just now
[病史詢問來源]:
    patient
[過去及個人病史]:
    denied any systemic disease

[藥物過敏史]: 如急診病歷首頁記載
[家族史]: not contributed
[現病史 Present Illness]:
    alcohol drinking+
    found outside a hotel
    with facial swelling, right chest wall pain told
    d

In [ ]:
model.save_pretrained_merged("model_mna", tokenizer, save_method = "merged_16bit",)

Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 5.7G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 30.82 out of 50.99 RAM for saving.


100%|██████████| 32/32 [00:40<00:00,  1.26s/it]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...
Done.
